In [25]:
%pip install openAI
%pip install langchain
%pip install typing-inspect==0.8.0 typing_extensions==4.5.0
%pip install pydantic==1.10.8

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [26]:
import openai
from langchain import *
from pydantic import *
import os 
import pandas as pd 
with open("secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1] #for some reason there's a new line, so we remove it



# Let's try LangChain

Literally just grabbed the example from the website and modified. still have no idea how this works but whatever (for now) 


## Putting everything together

In [31]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List



# Define your desired data structure.
class extracted_data(BaseModel):
    descriptions: list = Field(description="The descriptions for the shape. This should be a list of one or more strings.")
    
    





input =pd.read_csv("data/pre/thick_2.csv")

parser = PydanticOutputParser(pydantic_object=extracted_data)

info_query_generic = "Here's a partial transcript of someone describing an image to someone else. Extract verbatim the parts of the text that describe of the image. \
Do not include fillers like 'it looks like' or 'sorta'  \
  or conversational words that aren't descriptive. \n\n \
  As an example, if the transcript was 'It looks like a magician, and, uhh, I think he's got a rabbit.', the response would be ['a magician','he's got a rabbit'].\n\n \
     As an example, if the transcript was 'yes, sorry, big triangle is on the right, like an arm', the response would be ['big triangle is on the right, like an arm'].\n\n \
      Return just the description in list format. Here's the text:\n\n "
#model_name = "gpt-3.5-turbo"
model_name="gpt-4"
temperature = 0
model = ChatOpenAI(temperature=temperature)


def code_data_with_langchain(df, model, parser, info_query_generic): 
  langchain_prompt = PromptTemplate(
    template="{query}\n",
    input_variables=["query"]
  )
  system_prompt = "You are a competent researcher who answers user queries."

  res = []
  #for index in range(2):
  for index, _ in df.iterrows():
    print(index)
    info_query = info_query_generic + df["text"][index] + "\n\n"
    _input = langchain_prompt.format_prompt(query=info_query)
    #print(_input)
    full_prompt=[SystemMessage(content=system_prompt), HumanMessage(content=_input.to_string())]
    #print(full_prompt)
    output = model(full_prompt)
    #parsed_output = parser.parse(output)
    string_output=output.content
    print(string_output)
    res.append(string_output)
  return res


res = code_data_with_langchain(input, model, parser,  info_query_generic)

output=input
output['gpt_out']=res
output.to_csv("data/post/thick_2.csv")





0
['diamond head', 'bent legs']
1
['diamond and one triangle head', 'triangle face pointing towards left']
2
['diamond head', 'rectangle body', 'arm towards right', 'rectangle like body']
3
['diamond head', 'arm right', 'looks like bending']
4
['diamond head', 'long legs', 'looks like sitting']
5
['diamond head, no arms, or legs']
6
['diamond head', 'two legs', 'one leg lifting', 'zombie']
7
['head - square and two triangles', 'body triangle']
8
['diamond head', 'arms facing left', 'looks like wearing gown']
9
['the head has square', 'two triangle', 'long neck', 'has neck', 'square is upper left side of the neck', 'neck']
10
['skiing']
11
['open arms']
12
['sitting']
13
['open are', 'arm']
14
['no arm no legs, facing right']
15
['zombie']
16
['skiing']
17
['two triangles', 'one square on the head', 'body triangle']
18
['facing left', 'diamond head', 'wearing gown']
19
['neck!', 'long neck', 'had before']
20
['square and one triangle head, point face, facing left']
21
['diamond head, fa

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 13 Jul 2023 19:31:25 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e63ea9ad9541701-SJC', 'alt-svc': 'h3=":443"; ma=86400'}.


['low dancing figure', 'large leg outstretched to left', 'high shoulder on right']
335
['seated slumped pregnant figure, leg and foot visible on left']
336
['happy dancing figure', 'big shoulder on left', 'little kicked out leg on left']
337
['slumped sitting pregnant figure', 'head bowed to left', 'leg and foot visible']
338
['half kneeling figure', 'holding book to right', 'facing right']
339
['angel, arms up, flying to the left']
340
['boxy figure', 'pointy shoulder', 'pointy foot visible']
341
['double headed kneeling figure, facing right, holding book']
342
['ice skater going to the left', 'foot stretched out to right']
343
['straight backed sitting figure', 'head bowed down facing right', 'elaborate headwear']
344
['standing figure facing left, holding book']
345
['kneeling figure', 'head dipped to right with loudspeaker']
346
['sad and lonely figure', 'knees to right', 'no feet or arms visible']
347
['Stretched out dancer', 'large leg on left', 'high shoulder on right']
348
['st

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 13 Jul 2023 19:37:11 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e63f31218de9818-SJC', 'alt-svc': 'h3=":443"; ma=86400'}.


['kneeling facing right holding something out']
385
['sitting with their legs stuck out to the left']
386
['kneeling with both arms sticking out']
387
['balancing on one leg facing left']
388
['standing on one leg', 'one leg kicked forward']
389
['with two diamonds stacked on top']
390
['two triangles hanging off the right side']
391
['wrapped up tight']
392
['standing straight', 'holding something out to the left']
393
['blocky and pointy']
394
['leaning back with arms in the air']
395
['rabbit ears facing right']
396
['balancing on one leg with a leg kicked to the left']
397
['two triangles hanging off the right side']
398
['leaning back with arms in the air']
399
['rabbit ears facing right']
400
['two diamonds stacked on top']
401
['wrapped up tight']
402
['balancing on one leg with a leg stuck out to the right']
403
['standing straight facing left']
404
['sitting facing left']
405
['blocky and kneeling']
406
['kneeling with both arms out']
407
['kneeling facing right']
408
['leanin

...

['pointy mouse nose, facing left']
2020
['balancing on one leg facing left', 'leg sticking out on right']
2021
['bunny facing right']
2022
['facing right', 'holding plate', 'crouching abit', 'not kneeling', 'knees bent abit']
2023
['facing left', 'looks like a bear', 'with square on back']
2024
['facing left', 'looks like bunny but with square ear', 'tiny nose', 'long leg sticking out to left']
2025
['body is square but missing right side abit, straight down left side']
2026
['square body, missing some of right side']
2027
['diamond head in middle', 'body is same down both side then goes off to the right at bottom', 'baby shape', 'in sleeping bag']
2028
['facing right', 'crouching abit', 'not kneeling']
2029
['facing left', 'someone sat on floor knees bent']
2030
['pointy mouse nose, facing left']
2031
['balancing on one leg', 'facing left']
2032
['facing left', 'holding out a plate', 'straight across bottom', 'down right side']
2033
['waving arms in air']
2034
['facing left long and f

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


['two headed beast', 'right head is human and facing to our right', 'left head is like a bird with beak pointing down and to our left', 'One leg is raised and pointing to our left']
2388
['a sitting figure facing to our right, you cant see all their legs']
2389
['kneeling woman facing our right', 'they have shoulder pads', 'their legs are stretched behind them to our left']
2390
['Blocky figure', 'the shoulder on our right is pointed', 'the left shoulder is square']
2391
['figure with both arms raised up', 'figure is leaning to our left']
2392
['sitting girl facing to our right with bunny ears']
2393
['kneeling girl facing to our right', 'arms stretched out to our right', 'feet pointing to our left and upwards']
2394
['a twisted figure', 'legs pointing to our left and facing to our left', 'two odd bits pointing to our right']
2395
['a judo kick', 'one standing leg with toes pointing our left', 'kicking foot to our right']
2396
['woman in kimono pointing to our left with droopy sleeves'

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


['broken square']
2690
['man sitting, reading', 'Legs tilted towards the right']
2691
['standing man reading close to chest']
2692
['dancing woman']
2693
['the bowling', 'both hands sideways', 'one leg parallel']
2694
['sitting reading knee']
2695
['woman sitting more like kneeling with a book close to chest']
2696
['mountain top shape']
2697
['front action type', 'not parallel', 'one leg slightly up tilted']
2698
['woman sitting kneeling reading book']
2699
['mountain top shape']
2700
['man standing reading book close to chest']
2701
['crying woman']
2702
['man bowling', 'both hands sideways', 'One leg full up like parallel to the ground', 'Other firmly on ground']
2703
[]
2704
['dancing girl']
2705
There is no description of the image in the given text.
2706
['sitting reading on knee']
2707
['This is what you are confusing between two', 'This is not sideways but like throwing something', 'One leg slightly raised', 'Head appears to be downwards']
2708
['sitting reading man', 'Legs til